In [1]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd
import fitz
import os
import boto3
import io
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

F:\anaconda3\envs\python_3_7\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



In [2]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [252]:
Ima = "Makro Invoices/Makro Invoices/New/FD.- nominas de Marzo_page4.png"

In [253]:
def find_amounts(text):
    amounts = re.findall(r'\d+\.\d{2}\b', text)
    floats = [float(amount) for amount in amounts]
    unique = list(dict.fromkeys(floats))
    if len(unique)==0:
        return 0
    else:
        return max(unique)

In [254]:
dat = {"NIF":'',
      "seguridad_social": '',
      "dni":'',
      "seguridad_social1": '',
      "nombre": '',
      "nombre1": '',
      "categoria": '',
      "antiguedad": '',
      "periodo": '',
      "dias": '',
      "devengado": ''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
for idx, a in enumerate(text_list):
    if(a.strip() != ""):
        a = a.replace(',','.')
        if (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            print(b)
            dat['NIF'] = b[0]
        if (re.search("[0-9]{2}/[0-9]{7}-[0-9]{2}",a)):
            b = re.findall("[0-9]{2}/[0-9]{7}-[0-9]{2}",a)
            print(b)
            dat['seguridad_social'] = b[0]
        if (re.search("[0-9]{2}/[0-9]{8}-[0-9]{2}",a)):
            b = re.findall("[0-9]{2}/[0-9]{8}-[0-9]{2}",a)
            print(b)
            dat['seguridad_social1'] = b[0]
        if (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            print(b)
            dat['dni'] = b[0]

['B38339362']
['38/1010368-91']
['46262065A']
['38/10830624-63']


In [255]:
dat

{'NIF': 'B38339362',
 'seguridad_social': '38/1010368-91',
 'dni': '46262065A',
 'seguridad_social1': '38/10830624-63',
 'nombre': '',
 'nombre1': '',
 'categoria': '',
 'antiguedad': '',
 'periodo': '',
 'dias': '',
 'devengado': ''}

In [256]:
#find_amounts(text_list)
find_amounts(text.replace(',','.'))

275.34

In [257]:
text_list

['GARCIA BELTRAN, RAUL',
 '',
 'cL VERA DE ARRIBA 32',
 '38626 ARONA',
 'SC DE TENERIFE',
 'NIF. B38339362 784',
 'EMPRESA DOMICILIO N NS. SS.',
 'FERRETERIA DORTA SL CL CHINDIA, 10 38/1010368-91',
 'TRABAJADORIA CATEGORIA N°MATRIC. ANTIGUEDAD DNL',
 'IGARCIA BELTRAN, RAUL MOZO 19 JUL 21 46262065A',
 'N° AFILIAGION. S.S._[TARIFA|COD.CT| SECCION | _NRO. PERIODO TOT. DAS',
 '38/10830624-63 10 [189 123 [MENS 01 MAR 22 a 31 MAR 22 30',
 '‘CUANTIA PRECIO CONCEPTO. DEVENGOS | _DEDUCCIONES',
 '30,00 33,333 1] *Salario Base 1.000,00',
 '30] *P.P.Paga Extra 166,67',
 '995 COTIZACION CONT.COMU 4,70 54,83',
 '996 COTIZACION FORMACION 0,10 1,17',
 '997 COTIZACION DESEMPLEO 1,55 18,08',
 '999 TRIBUTACION I.R.P.F. 1,44 16,80',
 'REM.TOTAL | __P.P.EXTRAS BASE S.S. BASEA.T.Y DES. | BASELRP.F._| T. DEVENGADO| T. A DEDUCIR',
 '1.166, 67 1.166, 70 1.166, 70 1.166,67 | 1.166,67 90,88',
 '* Percepciones Salariales sujetas a Cot. S.S. - Percepciones no Salariales excluidas Cot. S.S.',
 'FECHA ‘SELLO EMPRESA

In [258]:
im = Image.open(Ima)

In [259]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [260]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [261]:
len(dataframes)

6

In [262]:
d1 = dataframes[0]
d2 = dataframes[1]
d3 = dataframes[2]
if len(dataframes)>3:
    d4 = dataframes[3]
if len(dataframes)>4:
    d5 = dataframes[4]
if len(dataframes)>5:
    d6 = dataframes[5]

In [263]:
d1.head(20)

,EMPRESA,DOMICILIO,N° INS. S.S.
0,FERRETERIA DORTA SL,"CL CHINDIA, 10",38/1010368-91


In [264]:
if 'EMPRESA' in d1.columns or 'DOMICILIO' in d1.columns:
    print('HI')
    for i in d1['EMPRESA']:
        nombre = i
    dat["nombre"] = nombre

HI


In [265]:
#dat["nombre"] = d1['EMPRESA']

In [266]:
dat

{'NIF': 'B38339362',
 'seguridad_social': '38/1010368-91',
 'dni': '46262065A',
 'seguridad_social1': '38/10830624-63',
 'nombre': 'FERRETERIA DORTA SL',
 'nombre1': '',
 'categoria': '',
 'antiguedad': '',
 'periodo': '',
 'dias': '',
 'devengado': ''}

In [267]:
d1['EMPRESA']

0    FERRETERIA DORTA SL
Name: EMPRESA, dtype: object

In [268]:
d2.head()

,TRABAJADOR/A,CATEGORIA,N°MATRIC,ANTIGUEDAD,D.N.I.
0,"GARCIA BELTRAN, RAUL",MOZO,,19 JUL 21,46262065A


In [269]:
if 'TRABAJADOR/A' in d2.columns or 'CATEGORIA' in d2.columns:
    for i in d2['TRABAJADOR/A']:
        nombre1 = i
    dat['nombre1'] = nombre1
    for j in d2['CATEGORIA']:
        categoria = j
    dat['categoria'] = categoria
    for k in d2['ANTIGUEDAD']:
        antigue = k
    dat['antiguedad'] = antigue
    

In [270]:
d3.head()

,N° AFILIACION. S.S.,TARIFA,COD.CT,SECCION,NRO.,PERIODO,TOT. DIAS
0,38/10830624-63,10,189,,123,MENS 01 MAR 22 a 31 MAR 22,30


In [271]:
if 'NRO.' in d3.columns or 'PERIODO' in d3.columns or 'TOT. DIAS' in d3.columns:
    for i in d3['PERIODO']:
        period = i
    dat["periodo"] = period
    for j in d3['TOT. DIAS']:
        dias = j
    dat['dias'] = dias
    

In [272]:
if 'REM. TOTAL' in d4.columns or 'T. DEVENGADO' in d4.columns:
    for i in d4['T. DEVENGADO']:
        devengado = i.replace(',','.')
    dat["devengado"] = devengado

if 'REM. TOTAL' in d5.columns or 'T. DEVENGADO' in d5.columns:
    for i in d5['T. DEVENGADO']:
        devengado = i.replace(',','.')
    dat["devengado"] = devengado

In [273]:
dat

{'NIF': 'B38339362',
 'seguridad_social': '38/1010368-91',
 'dni': '46262065A',
 'seguridad_social1': '38/10830624-63',
 'nombre': 'FERRETERIA DORTA SL',
 'nombre1': 'GARCIA BELTRAN, RAUL',
 'categoria': 'MOZO',
 'antiguedad': '19 JUL 21',
 'periodo': 'MENS 01 MAR 22 a 31 MAR 22',
 'dias': '30',
 'devengado': '1.166.67'}

In [277]:
dataframes[3]

,CUANTIA,PRECIO,,CONCEPTO,DEVENGOS,DEDUCCIONES
0,"30,00","33,333",1,*Salario Base,"1.000,00",
1,,,30,*P Paga Extra,"166,67",
2,,,995,"COTIZACION CONT.COMU 4,70",,"54,83"
3,,,996,"COTIZACION FORMACION 0,10",,"1,17"
4,,,997 999,"COTIZACION DESEMPLEO 1,55 TRIBUTACION I.R.P.F....",,"18,08 16,80"
5,,,,,,


In [278]:
im = Image.open(Ima) 
        # Size of the image in pixels (size of orginal image) 
        # (This is not mandatory) 
width, height = im.size 
print(im.size)
        # Setting the points for cropped image 
left = 605
top = 900
right = 1675
bottom = 1500
# Cropped image of above dimension 
        # (It will not change orginal image) 
im1 = im.crop((left, top, right, bottom)) 
         # Shows the image in image viewer 
im1.save("Makro Invoices/Makro Invoices/New/pp2.jpg")
img  = cv2.imread('Makro Invoices/Makro Invoices/New/pp2.jpg', cv2.COLOR_RGB2GRAY)
tex=(pytesseract.image_to_string(img))
text_list = tex.split("\n")
text_list = text_list
print(text_list)

(1190, 1684)
['"DES. BASE LR.P.F. T. DEVENGADO| T. A DEDUCIR', '36,70 1.166, 67 1.166,67 90,88', '~ Percepciones no Salariales excluidas Cot. S.S.', 'RECBI', '', 'LIQUIDO A PERCIBIR', '', '1.675; 79', '', 'DE RECAUDACION CONJUNTA Y APORTACION DE LA EMPRESA', '', 'BASE', '', '166,70', '166,70', '166,70', '', 'APORTACION EMPRESA RIAL}', '', '']


In [279]:
#find_amounts(text_list)
find_amounts(tex.replace(',','.'))

166.7